In [1]:
import os
import json

# print(data_json[0])

def parse_children(list_children,x_pattern,y_pattern,result):
    # result_children = []
    for i_child in list_children:
        type_child = i_child["type"]
        try:
            size_child = i_child["data"]["size"]
        except:
            size_child = None
        if type_child == "DIVIDER":
            sub_x = abs(i_child['data']['to']['x'] - i_child['data']['from']['x'])
            sub_y = abs(i_child['data']['to']['y'] - i_child['data']['from']['y'])
            if sub_x <= 1:
                width_child = 1
                height_child = sub_y
            if sub_y <= 1:
                width_child = sub_x
                height_child = 1
        else:
            try:
                width_child = i_child["data"]["width"]
                if width_child == None:
                    width_child = 0
            except:
                width_child = 0
            try:
                height_child = i_child["data"]["height"]
                if height_child == None:
                    height_child = 0
            except:
                height_child = 0
        x_child = i_child["data"]["position"]["x"] + x_pattern
        y_child = i_child["data"]["position"]["y"] + y_pattern
        if type_child == "CONTAINER":
            children = i_child["children"]
            result = parse_children(children,x_child,y_child,result)
        else:
            #x_c = x_child + width_child/2
            #y_c = y_child - height_child/2
            #result.append(f"{type_child}({x_c},{y_c})")
            if 'text' in i_child["data"]:
                result.append({
                    "type": type_child,
                    "text": i_child["data"]['text'],
                    "width": width_child,
                    "height": height_child,
                    "x": x_child,
                    "y": y_child,
                    "size": size_child
                })
            else:
                result.append({
                    "type": type_child,
                    "width": width_child,
                    "height": height_child,
                    "x": x_child,
                    "y": y_child,
                    "size": size_child
                })
    return result

def resize_component_pattern(data):
    list_pattern = []
    for i_pattern in data:
        name_pattern = i_pattern["name"]
        id_pattern = i_pattern['id']
        data_pattern = i_pattern["data"]
        width_pattern = data_pattern["data"]["width"]
        height_pattern = data_pattern["data"]["height"]
        x_pattern = data_pattern["data"]["position"]["x"]
        y_pattern = data_pattern["data"]["position"]["y"]
        list_children = data_pattern["children"]
        children_info = []
        children_info = parse_children(list_children,x_pattern,y_pattern,children_info)
        # print(children_info)
        list_pattern.append({
            "name_pattern": name_pattern,
            'id_pattern': id_pattern,
            "width_pattern": width_pattern,
            "height_pattern": height_pattern,
            "x_pattern": x_pattern,
            "y_pattern": y_pattern,
            "info_children": children_info
        })
        # break
        # print(list_pattern)
    return list_pattern


def translate_component(file_json):
    f = open(file_json)
    data_json = json.load(f)
    json_data = resize_component_pattern(data_json)
    #Lay toa do tuyet doi cua component
    for i in range(len(json_data)):
        a = 0 - json_data[i]['x_pattern']
        b = 0 - json_data[i]['y_pattern']
        json_data[i]['x_pattern'] = 0
        json_data[i]['y_pattern'] = 0
        for j in range(len(json_data[i]['info_children'])):
            json_data[i]['info_children'][j]['x'] = a + json_data[i]['info_children'][j]['x']
            json_data[i]['info_children'][j]['y'] = b + json_data[i]['info_children'][j]['y']
    
    #Tinh x_center y_center      
    for i in range(len(json_data)):
        for j in range(len(json_data[i]['info_children'])):
            json_data[i]['info_children'][j]['x'] = (json_data[i]['info_children'][j]['x'] + json_data[i]['info_children'][j]['width']) / 2
            json_data[i]['info_children'][j]['y'] = (json_data[i]['info_children'][j]['y'] + json_data[i]['info_children'][j]['height']) / 2
        
    return json_data

def type_and_position_of_pattern(file_json):
    patterns = translate_component(file_json)
    list_patterns = []
    for i in range(len(patterns)):
        children = []
        for j in range(len(patterns[i]['info_children'])):
            temp = patterns[i]['info_children'][j]['type']\
            + '(' + str(patterns[i]['info_children'][j]['x'])\
            + ',' + str(patterns[i]['info_children'][j]['y']) + ')'
            children.append(temp)
        pattern = {
            'id_pattern' : patterns[i]['id_pattern'],
            'width_pattern': patterns[i]['width_pattern'],
            'height_pattern': patterns[i]['height_pattern'],
            'info_children' : children
        }
        list_patterns.append(pattern)
    return list_patterns
list_patterns = translate_component('patterns.json')
# export json
with open('position_center.json', 'w') as f:
    json.dump(list_patterns, f)